Following guide from [lifelib](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points.html)

# [Generating model points for with duration](https://lifelib.io/libraries/notebooks/basiclife/generate_model_points_with_duration.html)

In [39]:
import numpy as np
from numpy.random import default_rng  # Requires NumPy 1.17 or newer

rng = default_rng(12345)

# Number of Model Points
MPCount = 10000

# Issue Age (Integer): 20 - 59 year old
age_at_entry = rng.integers(low=20, high=60, size=MPCount)

# Sex (Char)
Sex = [
    "M",
    "F"
]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=MPCount)), np.dtype('<U1'))

# Policy Term (Integer): 10, 15, 20
policy_term = rng.integers(low=0, high=3, size=MPCount) * 5 + 10


# Sum Assured (Float): 10000 - 1000000
sum_assured = np.round((1000000 - 10000) * rng.random(size=MPCount) + 10000, -3)

# Duration in month (Int): -36 < Duration(mth) < Policy Term in month
duration_mth = np.rint((policy_term + 3) * 12 * rng.random(size=MPCount) - 36).astype(int)

# Policy Count (Integer): 1
policy_count = np.rint(100 * rng.random(size=MPCount)).astype(int)

In [31]:
import pandas as pd

attrs = [
    "age_at_entry",
    "sex",
    "policy_term",
    "policy_count",
    "sum_assured",
    "duration_mth"
]

data = [
    age_at_entry,
    sex,
    policy_term,
    policy_count,
    sum_assured,
    duration_mth
]

model_point_table = pd.DataFrame(dict(zip(attrs, data)), index=range(1, MPCount+1))
model_point_table.index.name = "policy_id"
model_point_table

,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth
policy_id,,,,,,
1,47,M,10,86,622000.0,1
2,29,M,20,56,752000.0,210
3,51,F,10,83,799000.0,15
4,32,F,20,72,422000.0,125
5,28,M,15,99,605000.0,55
...,...,...,...,...,...,...
9996,47,M,20,25,827000.0,157
9997,30,M,15,81,826000.0,168
9998,45,F,20,10,783000.0,146


# [Create Premium Table](https://lifelib.io/libraries/notebooks/basiclife/create_premium_table.html)

In [32]:
%pip install modelx lifelib -q

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import lifelib
import shutil

# set to True if you want to remove existing cluster folder
REMOVE_CLUSTER = True

#check if there is a folder named "cluster" in the current directory
if os.path.exists("cluster") & REMOVE_CLUSTER:
    shutil.rmtree("cluster")

lifelib.create("cluster", r"./cluster")

In [34]:
import itertools
import modelx as mx
import pandas as pd

In [35]:
space.model_point_table = model_point_table

In [36]:
@mx.defcells
def premium_rate():
    return (1 + loading_prem()) * net_premium_pp()

In [38]:
model_point_table['premium'] = space.premium_rate()

# Distorting data for preso

In [ ]:
#Add noise
for col in model_point_table.columns:
    model_point_table.loc[model_point_table.sample(frac=0.1).index, col] = pd.np.nan

In [ ]:
distorted_names = ['UUid ',
 'Age At Entry',
 'Sex ',
 'Policy Term  ',
 'Policy Count',
 'Sum Assured ',
 'Issue date',
 'Payment freq',
 'Payment_term',
 'Premium']

rename_dict = dict()
for i in range(0,len(distorted_names)):
  rename_dict.update({list(model_point_table.columns)[i]:distorted_names[i]})
  
model_point_table = model_point_table.rename(columns=rename_dict)
model_point_table.columns

In [ ]:
model_point_table.to_excel("model_point.xlsx")

In [ ]:
space.premium_pp.formula

In [ ]:
df = pd.DataFrame(itertools.product(range(20, 60), [10, 15, 20]), columns=['age_at_entry', 'policy_term'])
df.index += 1
df.index.name = 'point_id'
df["sum_assured"] = 1
df

In [ ]:
len(space.premium_rate())

In [ ]:
premium_rate_table = df.copy()
space.model_point_table = df
premium_rate_table["premium_rate"] = space.premium_rate()
del premium_rate_table["sum_assured"]
premium_rate_table.set_index(["age_at_entry", "policy_term"], inplace=True)
premium_rate_table["premium_rate"]   # As Series

In [ ]:
model_point_table

# Additional Component for IAA preso